<a href="https://colab.research.google.com/github/valentinaandrade/constituyentes/blob/main/selenium_apunte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Web Scraping con Selenium en Python**


Es una herramienta que permite simular acciones humanas en un navegador, lo que significa que puede abrir una página web, hacer clic en elementos, enviar formularios, interactuar con JavaScript y extraer contenido.


## **Librerías**

Para este ejercicio utilizaremos las siguientes librerías. Para replicarlo es particularmente importante utilizar la misma versión de Selenium pues versión a versión suele haber una cantidad significativa de cambios.

- Python 3.8
- Selenium 4.15.2
- Numpy 1.24.4
- Pandas 2.0.3

Aquí el código para instalar una versión específica de las librerías a través de `pip`

In [ ]:
#!pip install selenium==4.15.2

## **Drivers**

Selenium utiliza un explorador web simulando la interacción humana. Para esto utiliza `drivers`, controlando a través de estos la dinámica de cada explorador. Hay disponibles para Chrome, Firefox, Edge, Internet Explorer y Safari. Puedes elegir el que más te acomode, sin embargo, te recomiendo utilizar Firefox pues tiene una mayor compatibilidad en relación a los drivers disponibles.

A continuación te dejo los enlaces para descargar dos de los más utilizados:

- **Firefox**: https://github.com/mozilla/geckodriver/releases.
- **Chrome**: https://chromedriver.chromium.org/downloads. Se tiene compatibilidad para la versión 114.0.5735.90 y anteriores.

Puedes probar también el uso de forma automática:

In [ ]:
# Opción "automática"

# options = webdriver.FirefoxOptions()
# driver = webdriver.Firefox(options=options)

# service = Service(ChromeDriverManager().install())
# driver = webdriver.Chrome(service=service)

Cargamos las librerías :

In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException  # Verificar que las excepciones estén importadas correctamente
import time
import pandas as pd
import re

Especificamos la ruta al archivo driver. En este caso `geckodriver` es un archivo sin extensión, no una carpeta.

In [ ]:
path = '/Users/jan/Desktop/UAI/Clase Selenium/Drivers/geckodriver'
service = Service(path)

En el siguiente paso se abrirá la ventana de Firefox indicando que es una sesión automatizada. De todos modos puedes interactuar con el explotador.

 - En Mac abrir "Configuración del sistema", sección de "Privacidad y seguridad" y permitir la ejecución del driver.

In [ ]:
driver = webdriver.Firefox(service=service)

### **Antes de continuar, un repaso**

**1. Iniciar el WebDriver**
```
from selenium import webdriver
driver = webdriver.Firefox()
# driver = webdriver.Firefox(executable_path=r'ruta/a/tu/geckodriver')
```

**2. Navegar a una página web**
```
driver.get("https://www.emol.com/")
```

**3. Localizar un elemento por ID**
```
from selenium.webdriver.common.by import By
elem = driver.find_element(By.ID,'frase_busqueda')
```

**4. Localizar un elemento por nombre**
```
elem = driver.find_element(By.NAME,'frase_busqueda')
```
**5. Localizar un elemento por clase CSS**
```
elem = driver.find_element(By.CLASS_NAME,"frase_input_bus")
```

**6. Localizar un elemento por selector CSS**
```
elem = driver.find_element(By.CSS_SELECTOR,"#frase_busqueda")
```

**7. Localizar un elemento por XPath**
```
elem = driver.find_element(By.XPATH,'//*[@id="frase_busqueda"]')
```

**8. Localizar varios elementos**
```
elements = driver.find_elements(By.TAG_NAME,'h3')
```

**9. Obtener el texto de un elemento**
```
for element in elements:
    print(element.text)
```

**10. Obtener un atributo de un elemento**
```
elem = driver.find_element(
    By.ID, 'ucHomePage_cuNoticiasCentral_LinkTitulo'
)
attribute = elem.get_attribute('href')
```

**11.   Enviar teclas a un elemento**
```
from selenium.webdriver.common.keys import Keys
elem = driver.find_element(By.ID,'frase_busqueda')
elem.send_keys("Bullying")
elem.send_keys(Keys.RETURN)
```

**12.  Hacer clic en un elemento**
```
elem = driver.find_element(By.ID,'ingresarH')
elem.click()
```

**13.  Ejecutar un script de JavaScript**
```
driver.execute_script("return document.title")
```

**14.   Esperar a que un elemento esté presente**
```
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located(
        (By.ID, "listado-avisos")
    )
)
```

**14. Cambiar entre ventanas o pestañas**
```
# Abre una nueva pestaña
driver.execute_script("window.open('');")

# Cambia a la pestaña por índice
driver.switch_to.window(driver.window_handles[0])

# Navega a una nueva URL en la nueva pestaña
driver.get('https://www.google.com')

# Obtiene nombres de las pestañas
driver.window_handles

# Cambia a pestaña por nombre
driver.switch_to.window("9e01f567-cf4b-43ac-9598-70924d09c016")
```

**15. Manejar cookies**
```
# Obtener todas las cookies
driver.get_cookies()

# Agregar una cookie
driver.add_cookie({'name' : 'foo', 'value' : 'bar'})

# Eliminar una cookie
driver.delete_cookie('foo')
```

**16. Cerrar el navegador**
```
# Cierra la ventana actual
driver.close()
# Cierra todas las ventanas y finaliza la sesión de WebDriver
driver.quit()  
```

## **Ejemplo:** Empleos Públicos

In [ ]:
# 1. Cargamos la web
driver.get("https://www.empleospublicos.cl/")
# 2. Asignamos un tiempo fijo de espera
time.sleep(4)
# 3. Nos aseguramos que un elemento que marca la disponibilidad de la información esté disponible
element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located(
        (By.CLASS_NAME, "item")
    )
)
# 4. Almacenamos los objetos que contienen el título de los avisos y sus links
h3_elements = driver.find_elements(By.TAG_NAME, 'h3')
# 5. Generamos listas vacías que rellenaremos con la información disponible.
links = []
titles = []
# 6. Para cada título extraemos su información.
for h3 in h3_elements:
    a_elements = h3.find_elements(By.TAG_NAME, 'a')
    for a in a_elements:
        links.append(a.get_attribute('href'))
        titles.append(a.text)
# 7. Almacenamos todo en un data.frame
avisos = pd.DataFrame({
    'Title': titles,
    'Link': links
})

In [ ]:
avisos

,Title,Link
0,"Auxiliar de Servicio, Diurno, Banco Nacional d...",https://www.empleospublicos.cl/pub/convocatori...
1,ADMINISTRATIVO ATENCIÓN CERRADA,https://www.empleospublicos.cl/pub/convocatori...
2,ADMINISTRATIVO ATENCIÓN ABIERTA,https://www.empleospublicos.cl/pub/convocatori...
3,ADMINISTRATIVO CONTABILIDAD,https://www.empleospublicos.cl/pub/convocatori...
4,ENCARGADO DE ADQUISICIONES,https://www.empleospublicos.cl/pub/convocatori...
...,...,...
450,Encargada/o Jardines Infantiles JUNJI; Comuna ...,https://junji.myfront.cl/oferta-de-empleo/1414...
451,Encargada/o Jardines Infantiles JUNJI; Comuna ...,https://junji.myfront.cl/oferta-de-empleo/1414...
452,Encargada/o Jardines Infantiles JUNJI; Comuna ...,https://junji.myfront.cl/oferta-de-empleo/1414...
453,Encargada/o Jardines Infantiles JUNJI; Comuna ...,https://junji.myfront.cl/oferta-de-empleo/1414...


Entre los avisos hay algunos que redireccionan a plataformas distintas a la de Empleos Públicos. Excluiremos tales casos para evitar errores. Revisaremos la frecuencia de estos casos y sus links.

In [ ]:
not_useful = avisos[~avisos['Link'].str.startswith("https://www.empleospublicos.cl/pub/convocatorias")]
print('Avisos excluidos: '+str(len(not_useful)) + ".")
not_useful.Link

Avisos excluidos: 23.


432    https://cmfchile.trabajando.cl/trabajo/5688769...
433    https://junji.myfront.cl/oferta-de-empleo/1413...
434    https://junji.myfront.cl/oferta-de-empleo/1418...
435    https://junji.myfront.cl/oferta-de-empleo/1423...
436    https://junji.myfront.cl/oferta-de-empleo/1420...
437    https://junji.myfront.cl/oferta-de-empleo/1423...
438    https://junji.myfront.cl/oferta-de-empleo/1258...
439    https://junji.myfront.cl/oferta-de-empleo/1366...
440    https://junji.myfront.cl/oferta-de-empleo/1365...
441    https://junji.myfront.cl/oferta-de-empleo/1365...
442    https://junji.myfront.cl/oferta-de-empleo/1372...
443    https://jobs.iadb.org/es?utm_source=eepp&utm_m...
444    https://junji.myfront.cl/oferta-de-empleo/1394...
445    https://junji.myfront.cl/oferta-de-empleo/1399...
446    https://junji.myfront.cl/oferta-de-empleo/1410...
447    https://www.sii.cl/sobre_el_sii/en_postulacion...
448    https://junji.myfront.cl/oferta-de-empleo/1405...
449    https://junji.myfront.cl

Habiendo revisado la estructura de los links adecuados filtramos por aquellos que nos entregan información

In [ ]:
avisos = avisos[avisos['Link'].str.startswith("https://www.empleospublicos.cl/pub/convocatorias/convpostularavisoTrabajo")]
avisos.Link[0]

'https://www.empleospublicos.cl/pub/convocatorias/convpostularavisoTrabajo.aspx?i=112355&c=0&j=0&tipo=convpostularavisoTrabajo'

Al entrar a cada uno de los links extraeremos la información disponible, para eso partiremos con una prueba individual.

In [ ]:
driver.get("https://www.empleospublicos.cl/pub/convocatorias/convpostularavisoTrabajo.aspx?i=112387&c=0&j=0&tipo=convpostularavisoTrabajo")

In [ ]:
# Localiza el contenedor con el id 'lblAvisoTrabajoDatos'
data_container = driver.find_element(By.ID, 'lblAvisoTrabajoDatos')

# Extraer todos los elementos h3 y p dentro del contenedor
headers = data_container.find_elements(By.TAG_NAME, 'h3')
paragraphs = data_container.find_elements(By.TAG_NAME, 'p')

# Diccionario para almacenar la información tabulada
data = {}

# Recorre los headers y párrafos asumiendo que cada h3 es seguido por un p
for header, paragraph in zip(headers, paragraphs):
    key = header.text.strip()
    value = paragraph.text.strip()
    data[key] = value

# Convertir el diccionario a DataFrame
df = pd.DataFrame([data])

# Imprime el DataFrame
df


,Institución,Convocatoria,Nº de Vacantes,Área de Trabajo,Región,Ciudad,Tipo de Vacante
0,Ministerio del Interior y Seguridad Pública / ...,Médico Cirujano con especialidad en Urología,2,Salud,Región Metropolitana de Santiago,Santiago,Honorarios


**Iteramos el código, con cuatro precauciones:**

1. Existe la posibilidad de que la información varíe página a página.
2. Podemos no encontrar los elementos que buscamos y recibir errores por ello.
3. Si por un error se cae nuestro proceso y no hemos almacenado la información, la perderemos.
4. Es recomendable ir almacenando la información recolectada si trabajamos con grandes volúmenes de datos. Puede colapsar nuestra memoria RAM.

In [ ]:
columnas_conocidas = set()

for i in avisos.Link:
    print(i)
    time.sleep(3)

    try:
        driver.get(i)
        # Asegurar que el elemento esté presente antes de proceder
        WebDriverWait(driver, 7).until(
            EC.presence_of_element_located((By.ID, "lblAvisoTrabajoDatos"))
        )
        data_container = driver.find_element(By.ID, 'lblAvisoTrabajoDatos')
        headers = data_container.find_elements(By.TAG_NAME, 'h3')
        paragraphs = data_container.find_elements(By.TAG_NAME, 'p')

        data = {}
        for header, paragraph in zip(headers, paragraphs):
            key = header.text.strip()
            value = paragraph.text.strip()
            data[key] = value
            columnas_conocidas.add(key)  # Añadir nuevas columnas al conjunto

    except (NoSuchElementException, TimeoutException) as e:  # Manejo de ambos tipos de errores
        print(f"Error al procesar la página {i}: {e}, se llenará con NA.")
        data = {col: pd.NA for col in columnas_conocidas}

    # Crear DataFrame con todas las columnas conocidas
    df = pd.DataFrame([data])
    df = df.reindex(columns=columnas_conocidas, fill_value=pd.NA)

    # Si es la primera iteración, escribe con encabezado, de lo contrario, sin encabezado
    if i == avisos.Link.iloc[0]:
        df.to_csv('output2.csv', mode='w', index=False)
    else:
        df.to_csv('output2.csv', mode='a', header=False, index=False)

https://www.empleospublicos.cl/pub/convocatorias/convpostularavisoTrabajo.aspx?i=112355&c=0&j=0&tipo=convpostularavisoTrabajo
https://www.empleospublicos.cl/pub/convocatorias/convpostularavisoTrabajo.aspx?i=112462&c=0&j=0&tipo=convpostularavisoTrabajo
https://www.empleospublicos.cl/pub/convocatorias/convpostularavisoTrabajo.aspx?i=112463&c=0&j=0&tipo=convpostularavisoTrabajo
https://www.empleospublicos.cl/pub/convocatorias/convpostularavisoTrabajo.aspx?i=112464&c=0&j=0&tipo=convpostularavisoTrabajo
https://www.empleospublicos.cl/pub/convocatorias/convpostularavisoTrabajo.aspx?i=112465&c=0&j=0&tipo=convpostularavisoTrabajo
https://www.empleospublicos.cl/pub/convocatorias/convpostularavisoTrabajo.aspx?i=112744&c=0&j=0&tipo=convpostularavisoTrabajo
https://www.empleospublicos.cl/pub/convocatorias/convpostularavisoTrabajo.aspx?i=113126&c=0&j=0&tipo=convpostularavisoTrabajo
https://www.empleospublicos.cl/pub/convocatorias/convpostularavisoTrabajo.aspx?i=113529&c=0&j=0&tipo=convpostularaviso

In [ ]:
df = pd.read_csv('output2.csv')
df


,Ciudad,Nº de Vacantes,Área de Trabajo,Región,Tipo de Vacante,Institución,Convocatoria
0,Santiago,1.0,Salud,Región Metropolitana de Santiago,Contrata,Ministerio de Salud / Servicio de Salud Metrop...,"Auxiliar de Servicio, Diurno, Banco Nacional d..."
1,Quellón,19.0,Salud,Región de Los Lagos,Contrata,Ministerio de Salud / Servicio de Salud Chiloé...,ADMINISTRATIVO ATENCIÓN CERRADA
2,Quellón,26.0,Salud,Región de Los Lagos,Contrata,Ministerio de Salud / Servicio de Salud Chiloé...,ADMINISTRATIVO ATENCIÓN ABIERTA
3,Quellón,3.0,Salud,Región de Los Lagos,Contrata,Ministerio de Salud / Servicio de Salud Chiloé...,ADMINISTRATIVO CONTABILIDAD
4,Quellón,1.0,Salud,Región de Los Lagos,Contrata,Ministerio de Salud / Servicio de Salud Chiloé...,ENCARGADO DE ADQUISICIONES
...,...,...,...,...,...,...,...
328,Penco,1.0,Salud,Región del Biobío,Contrata,Ministerio de Salud / Servicio de Salud Talcah...,(866-2667) Administrativo(a) Unidad de ciclo l...
329,Talcahuano,1.0,Salud,Región del Biobío,Contrata,Ministerio de Salud / Servicio de Salud Talcah...,"(850-2666) Médico Medicina Familiar, 22 horas,..."
330,Rancagua,1.0,Area para cumplir misión institucional,Región del Libertador General Bernardo O'Higgins,Contrata,"Ministerio de Economía, Fomento y Turismo / Su...",COORDINADOR(A) REGIONAL
331,Santiago,1.0,Area para cumplir misión institucional,Región Metropolitana de Santiago,Contrata,Ministerio de Hacienda / Unidad de Análisis Fi...,FISCALIZADOR(A) - DIVISIÓN FISCALIZACIÓN Y CUM...
